In [1]:
import numpy as np
import json

In [64]:
x = np.array([0, 0, -0.068, -0.137, -0.137, -0.137, -0.162])
y = np.array([0, 0, 0, 0, 0.2, 0.39, 0.463])
z = np.array([0.12, 0.32, 0.509, 0.696, 0.7, 0.7, 0.687])
roll = np.array([0, 0, 0.349, 1.569, -1.57, -1.694, -1.694])
pitch = np.array([0, -0.349, 0, -1.222, -0.348, -0.362, -0.326])
yaw = np.array([0, 0, -1.571, -3.14, 0, 0.372, 0.372])

axes = np.array([[0, 0, 1], [0, -1, 0], [0, 0, 1], [0, 1, 0], [0, 0, 1], [0, -1, 0], [0, 0, 1]])
origins = np.array([x, y, z]).T
angles_rpy = np.array([roll, pitch, yaw]).T

origins

array([[ 0.   ,  0.   ,  0.12 ],
       [ 0.   ,  0.   ,  0.32 ],
       [-0.068,  0.   ,  0.509],
       [-0.137,  0.   ,  0.696],
       [-0.137,  0.2  ,  0.7  ],
       [-0.137,  0.39 ,  0.7  ],
       [-0.162,  0.463,  0.687]])

In [58]:
def rotate(angles_rpy, vector):
    gamma = angles_rpy[0]
    beta = angles_rpy[1]
    alpha = angles_rpy[2]
    
    sin = np.sin
    cos = np.cos
    
    a = np.array([[cos(alpha)*cos(beta), cos(alpha)*sin(beta)*sin(gamma) - sin(alpha)*cos(gamma), cos(alpha)*sin(beta)*cos(gamma) + sin(alpha)*sin(gamma)],
                 [sin(alpha)*cos(beta), sin(alpha)*sin(beta)*sin(gamma) + cos(alpha)*cos(gamma), sin(alpha)*sin(beta)*cos(gamma) - cos(alpha)*sin(gamma)],
                 [-sin(beta), cos(beta)*sin(gamma), cos(beta)*cos(gamma)]])
    
    b = a.dot(vector / np.linalg.norm(vector))
    
    return b / np.linalg.norm(b)


def build_jacobian(origins, angles_rpy, axes, offset=0):
    j0 = np.concatenate([np.cross([0, 0, 1], origins[offset]), [0, 0, 1]])
    j = np.array([np.concatenate([np.cross(rotate(angles_rpy[i], axes[i]), origin - origins[i]), rotate(angles_rpy[i], axes[i])]) for i, origin in enumerate(origins[offset+1:])])

    return np.concatenate([[j0], j]).T

In [59]:
jacobian = build_jacobian(origins, angles_rpy, axes, offset=1)

np.set_printoptions(precision=3, linewidth=120)
print(jacobian)

[[ 0.000e+00  0.000e+00 -3.760e-01 -1.879e-01 -1.333e-01 -1.320e-02]
 [ 0.000e+00 -6.800e-02 -0.000e+00  4.737e-04 -3.759e-03 -2.225e-05]
 [ 0.000e+00  0.000e+00 -1.370e-01 -6.839e-02  3.665e-01  2.493e-02]
 [ 0.000e+00  0.000e+00  0.000e+00 -3.420e-01  9.398e-01 -2.716e-04]
 [ 0.000e+00  0.000e+00 -1.000e+00  6.965e-05 -2.996e-04  1.000e+00]
 [ 1.000e+00  1.000e+00  0.000e+00  9.397e-01  3.418e-01  7.486e-04]]


In [60]:
jacobian_inv = np.linalg.inv(jacobian)
print(jacobian_inv)

[[ 2.274e+00  1.471e+01 -7.900e+00  3.099e+00  2.274e-01  1.000e+00]
 [ 2.887e-02 -1.471e+01 -9.731e-02 -1.678e-02  2.475e-03 -0.000e+00]
 [-1.255e+00 -0.000e+00 -4.567e+00  1.603e+00  9.773e-02 -0.000e+00]
 [-2.163e+00 -0.000e+00  7.519e+00 -3.239e+00 -2.169e-01 -0.000e+00]
 [-7.874e-01 -0.000e+00  2.735e+00 -1.142e-01 -7.860e-02 -0.000e+00]
 [-1.255e+00 -0.000e+00 -4.567e+00  1.604e+00  1.098e+00 -0.000e+00]]


In [61]:
force = np.concatenate([rotate(angles_rpy[-1], [0, 0, 1]), [0, 0, 0]])
torque = jacobian_inv.dot(force)

print('Force vector: {}'.format(force))
print('Joint torques: {}'.format(torque))

Force vector: [-0.324  0.939 -0.116  0.     0.     0.   ]
Joint torques: [ 13.989 -13.805   0.939  -0.174  -0.063   0.939]


In [54]:
-1.694 + 0.5 * np.pi

-0.12320367320510339